# Stanza Quickstart

This notebook demonstrates the basic workflow for using Stanza to tune quantum devices.

## 1. Setup and Imports

In [ ]:
import numpy as np

from stanza.models import DeviceConfig
from stanza.routines import RoutineRunner, routine

## 2. Define Device Configuration

In practice, you'd load this from a YAML file. Here we'll create it programmatically for demonstration.

In [ ]:
device_yaml = """
name: "Demo Quantum Device"

gates:
  G1: {type: BARRIER, control_channel: 1, v_lower_bound: -3.0, v_upper_bound: 3.0}
  G2: {type: PLUNGER, control_channel: 2, v_lower_bound: -3.0, v_upper_bound: 3.0}
  G3: {type: BARRIER, control_channel: 3, v_lower_bound: -3.0, v_upper_bound: 3.0}

contacts:
  SOURCE: {type: SOURCE, control_channel: 4, measure_channel: 1, v_lower_bound: -3.0, v_upper_bound: 3.0}
  DRAIN: {type: DRAIN, control_channel: 5, measure_channel: 2, v_lower_bound: -3.0, v_upper_bound: 3.0}

routines:
  - name: sweep_barrier
    parameters:
      gate: G1
      v_start: -2.0
      v_stop: 0.0
      n_points: 100

instruments:
  - name: qdac2-control
    type: CONTROL
    driver: qdac2
    ip_addr: 127.0.0.1
    slew_rate: 1.0

  - name: qdac2-measurement
    type: MEASUREMENT
    driver: qdac2
    ip_addr: 127.0.0.1
"""

# Parse configuration
import yaml

config = DeviceConfig.model_validate(yaml.safe_load(device_yaml))

## 3. Write Routines

Routines are decorated functions that receive a context object with access to resources and results.

In [ ]:
@routine
def sweep_barrier(
    ctx, gate="G1", v_start=-2.0, v_stop=0.0, n_points=100, contact="DRAIN"
):
    """Sweep a barrier gate and measure drain current."""
    device = ctx.resources.device

    # Generate voltage range
    voltages = np.linspace(v_start, v_stop, n_points)

    # Perform sweep
    v_data, i_data = device.sweep_1d(
        gate_electrode=gate, voltages=voltages.tolist(), measure_electrode=contact
    )

    return {"gate": gate, "voltages": v_data, "currents": i_data}

In [ ]:
@routine
def analyze_sweep(ctx, threshold=1e-9):
    """Analyze results from previous sweep routine."""
    # Get data from previous routine
    sweep_data = ctx.results.get("sweep_barrier")

    if not sweep_data:
        return {"error": "No sweep data found"}

    voltages = np.array(sweep_data["voltages"])
    currents = np.array(sweep_data["currents"])

    # Find where current exceeds threshold
    threshold_idx = np.where(np.abs(currents) > threshold)[0]

    if len(threshold_idx) > 0:
        threshold_voltage = voltages[threshold_idx[0]]
    else:
        threshold_voltage = None

    return {
        "max_current": float(np.max(np.abs(currents))),
        "threshold_voltage": threshold_voltage,
        "threshold": threshold,
    }

## 4. Create Runner and Execute

The `RoutineRunner` manages resources and executes routines.

In [ ]:
try:
    runner = RoutineRunner(configs=[config])
    print(f"Available routines: {runner.list_routines()}")
except Exception as e:
    print(f"Note: Device connection would be needed to run: {e}")

## 5. Run Routines

Execute individual routines or chains of routines.

In [ ]:
# Run sweep
result = runner.run("sweep_barrier", gate="G1", contact="DRAIN")
print(f"Sweep completed: {len(result['voltages'])} points")

# Run analysis on sweep results
analysis = runner.run("analyze_sweep", threshold=1e-9)
print(f"Analysis: {analysis}")

## Summary

Stanza provides:
- **Simple configuration** with YAML
- **Decorator-based routines** for clean, testable code
- **Resource management** through context objects
- **Result chaining** for complex workflows
- **Automatic logging** of all measurements

See the [documentation](https://github.com/conductorquantum/stanza) for more examples and API details.